In [1]:
import hashlib

hash_md5 = hashlib.md5()
with open("/home/carlos_rocha/Documentos/POC/EasyOCR/trainer/craft/pretrained_model/CRAFT_B3.pth", "rb") as f:
    for chunk in iter(lambda: f.read(4096), b""):
        hash_md5.update(chunk)
        
hash_md5.hexdigest()

'0b9fbbf6abb2029da646521dce5ba1e6'

In [ ]:
import numpy as np
from glob import glob
import cv2

In [ ]:
def generate_gaussian_map(self):
    circle_mask = self.generate_circle_mask()

    gaussian_map = np.zeros((self.init_size, self.init_size), np.float32)

    for i in range(self.init_size):
        for j in range(self.init_size):
            gaussian_map[i, j] = (
                1
                / 2
                / np.pi
                / (self.sigma ** 2)
                * np.exp(
                    -1
                    / 2
                    * (
                        (i - self.init_size / 2) ** 2 / (self.sigma ** 2)
                        + (j - self.init_size / 2) ** 2 / (self.sigma ** 2)
                    )
                )
            )

    gaussian_map = gaussian_map * circle_mask
    gaussian_map = (gaussian_map / np.max(gaussian_map)).astype(np.float32)

    gaussian_map_color = (gaussian_map * 255).astype(np.uint8)
    gaussian_map_color = cv2.applyColorMap(gaussian_map_color, cv2.COLORMAP_JET)
    return gaussian_map, gaussian_map_color

gaussian_map, gaussian_map_color = generate_gaussian_map()

In [ ]:
init_size = 200

def four_point_transform(bbox):
    """
    Using the bbox, standard 2D gaussian map, returns Transformed 2d Gaussian map
    """
    width, height = (
        np.max(bbox[:, 0]).astype(np.int32),
        np.max(bbox[:, 1]).astype(np.int32),
    )
    init_points = np.array(
        [
            [0, 0],
            [init_size, 0],
            [init_size, init_size],
            [0, init_size],
        ],
        dtype="float32",
    )

    M = cv2.getPerspectiveTransform(init_points, bbox)
    warped_gaussian_map = cv2.warpPerspective(gaussian_map, M, (width, height))
    return warped_gaussian_map, width, height

In [2]:
def load_img_gt_box(img_gt_box_path):
    lines = open(img_gt_box_path, encoding="utf-8").readlines()
    word_bboxes = []
    words = []
    for line in lines:
        box_info = line.strip().encode("utf-8").decode("utf-8-sig").split(",")
        box_points = [int(box_info[i]) for i in range(8)]
        box_points = np.array(box_points, np.float32).reshape(4, 2)
        word = box_info[8:]
        word = ",".join(word)
        # if word in do_not_care_label:
        #     words.append(do_not_care_label[0])
        #     word_bboxes.append(box_points)
        #     continue
        word_bboxes.append(box_points)
        words.append(word)
    return np.array(word_bboxes), words

In [ ]:
import math
import numpy as np

def pointAngle(Apoint, Bpoint):
    angle = (Bpoint[1] - Apoint[1]) / ((Bpoint[0] - Apoint[0]) + 10e-8)
    return angle

def pointDistance(Apoint, Bpoint):
    return math.sqrt((Bpoint[1] - Apoint[1])**2 + (Bpoint[0] - Apoint[0])**2)

def lineBiasAndK(Apoint, Bpoint):

    K = pointAngle(Apoint, Bpoint)
    B = Apoint[1] - K*Apoint[0]
    return K, B

def getX(K, B, Ypoint):
    return int((Ypoint-B)/K)

def sidePoint(Apoint, Bpoint, h, w, placehold, enlarge_size):

    K, B = lineBiasAndK(Apoint, Bpoint)
    angle = abs(math.atan(pointAngle(Apoint, Bpoint)))
    distance = pointDistance(Apoint, Bpoint)

    x_enlarge_size, y_enlarge_size = enlarge_size

    XaxisIncreaseDistance = abs(math.cos(angle) * x_enlarge_size * distance)
    YaxisIncreaseDistance = abs(math.sin(angle) * y_enlarge_size * distance)

    if placehold == 'leftTop':
        x1 = max(0, Apoint[0] - XaxisIncreaseDistance)
        y1 = max(0, Apoint[1] - YaxisIncreaseDistance)
    elif placehold == 'rightTop':
        x1 = min(w, Bpoint[0] + XaxisIncreaseDistance)
        y1 = max(0, Bpoint[1] - YaxisIncreaseDistance)
    elif placehold == 'rightBottom':
        x1 = min(w, Bpoint[0] + XaxisIncreaseDistance)
        y1 = min(h, Bpoint[1] + YaxisIncreaseDistance)
    elif placehold == 'leftBottom':
        x1 = max(0, Apoint[0] - XaxisIncreaseDistance)
        y1 = min(h, Apoint[1] + YaxisIncreaseDistance)
    return int(x1), int(y1)

def enlargebox(box, h, w, enlarge_size, horizontal_text_bool):

    if not horizontal_text_bool:
        enlarge_size = (enlarge_size[1], enlarge_size[0])

    box = np.roll(box, -np.argmin(box.sum(axis=1)), axis=0)

    Apoint, Bpoint, Cpoint, Dpoint = box
    K1, B1 = lineBiasAndK(box[0], box[2])
    K2, B2 = lineBiasAndK(box[3], box[1])
    X = (B2 - B1)/(K1 - K2)
    Y = K1 * X + B1
    center = [X, Y]

    x1, y1 = sidePoint(Apoint, center, h, w, 'leftTop', enlarge_size)
    x2, y2 = sidePoint(center, Bpoint, h, w, 'rightTop', enlarge_size)
    x3, y3 = sidePoint(center, Cpoint, h, w, 'rightBottom', enlarge_size)
    x4, y4 = sidePoint(Dpoint, center, h, w, 'leftBottom', enlarge_size)
    newcharbox = np.array([[x1, y1], [x2, y2], [x3, y3], [x4, y4]])
    return newcharbox

In [4]:
for img_gt_box_path in glob('./dataset/ch4_training_localization_transcription_gt/*.txt'):
    word_bboxes, words = load_img_gt_box(img_gt_box_path)
    break

In [ ]:
word_bboxes

map_h, map_w = score_map.shape
bbox = enlargebox(bbox, map_h, map_w, enlarge_size, horizontal_text_bool)

bbox_left, bbox_top = np.array([np.min(bbox[:, 0]), np.min(bbox[:, 1])]).astype(
    np.int32
)
bbox -= (bbox_left, bbox_top)
warped_gaussian_map, width, height = four_point_transform(
    bbox.astype(np.float32)
)

bbox_area_of_image = score_map[
    bbox_top : bbox_top + height, bbox_left : bbox_left + width,
]
high_value_score = np.where(
    warped_gaussian_map > bbox_area_of_image,
    warped_gaussian_map,
    bbox_area_of_image,
)
score_map[
    bbox_top : bbox_top + height, bbox_left : bbox_left + width,
] = high_value_score

array([[[ 121.,   71.],
        [ 183.,   71.],
        [ 183.,  115.],
        [ 121.,  115.]],

       [[ 193.,   71.],
        [ 203.,   71.],
        [ 203.,  115.],
        [ 193.,  115.]],

       [[ 211.,   71.],
        [ 422.,   71.],
        [ 422.,  113.],
        [ 211.,  113.]],

       ...,

       [[2379., 3432.],
        [2424., 3432.],
        [2424., 3476.],
        [2379., 3476.]],

       [[2247.,   73.],
        [2368.,   73.],
        [2368.,  107.],
        [2247.,  107.]],

       [[2381.,   72.],
        [2388.,   72.],
        [2388.,  107.],
        [2381.,  107.]]], shape=(204, 4, 2), dtype=float32)